In [1]:
import pandas as pd
import geopandas as gpd
import os
from pathlib import Path
import json
import dash
import re
import dash_leaflet as dl
import dash_leaflet.express as dlx
from dash_extensions.enrich import DashProxy, Input, Output, html,dcc
from dash_extensions.javascript import arrow_function, assign
from pathlib import Path
from shapely.ops import unary_union
from shapely.geometry import shape, mapping
from shapely.ops import unary_union


# Import data

## Import the csv

In [2]:
df = pd.read_csv('granular.csv')

In [3]:
df.head()

,Unnamed: 0,ID Kabupaten,Tanggal Kejadian,Kejadian,Kabupaten,Provinsi,ID Provinsi,ID Kecamatan,Kecamatan
0,0,6202,2025-12-04,BANJIR,Kotawaringin Timur,kalimantan_tengah,62,6202201.0,tualan hulu
1,1,3313,2025-12-04,TANAH LONGSOR,Karanganyar,jawa_tengah,33,3313170.0,jenawi
2,2,3213,2025-12-04,BANJIR,Subang,jawa_barat,32,3213030.0,cisalak
3,3,3213,2025-12-04,BANJIR,Subang,jawa_barat,32,1216010.0,salak
4,4,3213,2025-12-04,BANJIR,Subang,jawa_barat,32,5371010.0,alak


In [4]:
len(df)

107111

## Import the geojson

In [5]:
def remove_id_prefix(name):
    return re.sub(r'^id', '', name)

### Run this only if it's the first time 

In [6]:
# link = 'https://github.com/JfrAziz/indonesia-district'
# repo_name = link.split("/")[-1]
# !rm -rf {repo_name}
# !git clone {link}

In [7]:
# repo_path = repo_name
# for root, dirs, files in os.walk(repo_path,topdown=False):
#     for file in files:
#         if file.startswith('id'):
#             old_path = os.path.join(root, file)
#             new_name = remove_id_prefix(file)
#             new_path = os.path.join(root, new_name)
#             os.rename(old_path, new_path)
#             # print(f"File: {file} → {new_name}")

#     for dir_name in dirs:
#         if dir_name.startswith('id'):
#             old_path = os.path.join(root, dir_name)
#             new_name = remove_id_prefix(dir_name)
#             new_path = os.path.join(root, new_name)
#             os.rename(old_path, new_path)
#             # print(f"Dir: {dir_name} → {new_name}")

# Data Dashboard

## Mapping

### Mapping provinsi-id

In [8]:
provinsi_dict = df.set_index('Provinsi')['ID Provinsi'].to_dict()
provinsi_options = [
    {'label': prov.replace('_', ' ').title(),'value':prov} for prov in sorted(provinsi_dict.keys())
]

### Mapping Kejadian

In [9]:
all_kejadian = df['Kejadian'].unique()
kejadian_options = [
    {'label':kej, 'value':kej} for kej in sorted(all_kejadian)
]

## Extract Date

In [10]:
df['Year'] = pd.to_datetime(df['Tanggal Kejadian']).dt.year

min_year = int(df['Year'].min())
max_year = int(df['Year'].max())


## Importing geojson

### Create Geojson path mapping

In [11]:
geojson_path = Path('indonesia-district')

In [12]:
# create a function that will precompute all path for each id 

def precompute_path(isprovince:bool=True) -> dict:
    all_dict:dict = {}
    for subdir,dirs, files in os.walk(geojson_path):
        for file in files:
            file_path = subdir+os.sep+file

            if file_path.endswith('.geojson'):
                file_id = file.split('_')[0]
                if isprovince and len(file_id) == 2:
                    all_dict[file_id]=file_path
                elif not isprovince:
                    all_dict[file_id]=file_path

    return all_dict

# id_geojson_path = precompute_path()
all_geojson_path = precompute_path(False)


## Create province-cities mapping

In [13]:
def build_province_city_mapping(path_dict: dict) -> dict:
    province_city_map = {}
    
    for file_id in path_dict.keys():
        if len(file_id) == 4:
            province_id = file_id[:2]
            
            if province_id not in province_city_map:
                province_city_map[province_id] = []
            
            province_city_map[province_id].append(file_id)
    
    for province_id in province_city_map:
        province_city_map[province_id].sort()
    return province_city_map


In [14]:
province_city_map = build_province_city_mapping(all_geojson_path)

## Cache the geojson

In [15]:
def precompute_geojson(path_dict: dict) -> dict:
    geojson_cache = {}
    
    for file_id, file_path in path_dict.items():
        try:
            with open(file_path, 'r') as f:
                geojson_data = json.load(f)
            
            geometries = []
            for feature in geojson_data['features']:
                geom = shape(feature['geometry'])
                geometries.append(geom)
            
            merged_geom = unary_union(geometries)
            
            file_name = Path(file_path).stem 
            if '_' in file_name:
                # Extract name part and clean it up
                name_part = '_'.join(file_name.split('_')[1:])  # Everything after first underscore
                display_name = name_part.replace('_', ' ').title()
            else:
                display_name = file_id
            
            # Convert back to GeoJSON feature
            merged_feature = {
                'type': 'Feature',
                'properties': {
                    'name': display_name,  # Human-readable name
                    'id': file_id
                },
                'geometry': mapping(merged_geom)
            }
            
            # Store in cache
            geojson_cache[file_id] = {
                "type": "FeatureCollection",
                "features": [merged_feature]
            }
            
            print(f"✓ Cached: {display_name} (ID: {file_id})")
            
        except Exception as e:
            print(f"✗ Error loading {file_id}: {e}")
    
    print(f"Total cached: {len(geojson_cache)}")
    return geojson_cache
    
GEOJSON_CACHE = precompute_geojson(all_geojson_path)

✓ Cached: prov 34.geojson (ID: prov 34.geojson)
✓ Cached: prov 37 simplified.geojson (ID: prov 37 simplified.geojson)
✓ Cached: prov 37.geojson (ID: prov 37.geojson)
✗ Error loading kab 34.geojson: TopologyException: side location conflict at 133.61439082717118 -2.2361045469220069. This can occur if the input geometry is invalid.
✗ Error loading kab 37.geojson: TopologyException: side location conflict at 133.61439082717118 -2.2361045469220069. This can occur if the input geometry is invalid.
✓ Cached: prov 34 simplified.geojson (ID: prov 34 simplified.geojson)
✓ Cached: Bengkulu District (ID: 17)
✓ Cached: Pinang Raya (ID: 1703093)
✓ Cached: Batik Nau (ID: 1703071)
✓ Cached: Enggano (ID: 1703010)
✓ Cached: Ulok Kupai (ID: 1703092)
✓ Cached: Arma Jaya (ID: 1703061)
✓ Cached: Air Padang (ID: 1703073)
✓ Cached: Air Napal (ID: 1703051)
✓ Cached: Ketahun (ID: 1703090)
✓ Cached: Lais (ID: 1703070)
✓ Cached: Kerkap (ID: 1703050)
✓ Cached: Putri Hijau (ID: 1703100)
✓ Cached: Bengkulu Utara (I

### function for retriving instance of geojson more efficiently

In [16]:
def retrieve_instance_geojson(list_id: list, density_dict: dict = None) -> dict:
    all_features: list = []
    
    for each_id in list_id:
        # Get from cache instead of file system
        cached_geojson = GEOJSON_CACHE.get(each_id)
        
        if not cached_geojson:
            print(f"Warning: ID {each_id} not found in cache")
            continue
        
        # Deep copy to avoid modifying the cache
        geojson_copy = json.loads(json.dumps(cached_geojson))
        
        # Apply density value
        density_value = density_dict.get(each_id, 0) if density_dict else 100
        
        for feature in geojson_copy['features']:
            feature['properties']['density'] = density_value
        
        all_features.extend(geojson_copy['features'])
    
    return {
        "type": "FeatureCollection",
        "features": all_features
    }

## Panel Function

In [17]:
def get_info(feature=None):
    header = [html.H4("Indonesia Disaster Dashboard")]
    if not feature:
        return header + [html.P("Hover pada wilayah untuk melihat detail")]
    
    name = feature.get("properties", {}).get("name", "Unknown")
    density = feature.get("properties", {}).get("density", 0)
    
    return header + [
        html.B(name),
        html.Br(),
        f"{int(density):,} kejadian",
    ]

info = html.Div(
    children=get_info(),
    id="info",
    className="info",
    style={
        "position": "absolute", 
        "top": "10px", 
        "right": "10px", 
        "zIndex": "1000",
        "background": "white", 
        "padding": "15px", 
        "borderRadius": "5px",
        "boxShadow": "0 2px 5px rgba(0,0,0,0.2)"
    }
)


## Styling

In [18]:
# Define color classes and scale
classes = [0, 50, 100, 250, 500, 750, 1000, 1500, 2000]
colorscale = ["#FFFFCC", "#FFEDA0", "#FED976", "#FEB24C", "#FD8D3C", "#FC4E2A", "#E31A1C", "#BD0026", "#800026"]
style = dict(weight=0, fillOpacity=0.8)

# Create colorbar
ctg = ["{}+".format(cls) for cls in classes[:-1]] + ["{}+".format(classes[-1])]
colorbar = dlx.categorical_colorbar(
    categories=ctg, 
    colorscale=colorscale, 
    width=300, 
    height=30, 
    position="bottomleft"
)

# Geojson rendering logic (JavaScript)
style_handle = assign("""function(feature, context){
    const {classes, colorscale, style, colorProp} = context.hideout;
    const value = feature.properties[colorProp];
    for (let i = 0; i < classes.length; ++i) {
        if (value > classes[i]) {
            style.fillColor = colorscale[i];
        }
    }
    return style;
}""")

## Initialize Map

In [19]:
# Initialize with empty geojson
# initial_geojson_data = {"type": "FeatureCollection", "features": []}
initial_geojson_data = retrieve_instance_geojson(['12'])


In [20]:

# Create GeoJSON layer
geojson = dl.GeoJSON(
    data=initial_geojson_data,
    style=style_handle,
    zoomToBounds=True,
    zoomToBoundsOnClick=True,
    hoverStyle=arrow_function(dict(weight=5, color="#666", dashArray="")),
    hideout=dict(colorscale=colorscale, classes=classes, style=style, colorProp="density"),
    id="geojson",
)

# Create info panel
info = html.Div(
    children=get_info(),
    id="info",
    className="info",
    style={"position": "absolute", "top": "10px", "right": "10px", "zIndex": "1000",
           "background": "white", "padding": "15px", "borderRadius": "5px"},
)

## Control UI

In [21]:
controls = html.Div([
    html.Div([
        html.Label("Pilih Provinsi", style={'fontWeight': 'bold', 'marginBottom': '5px'}),
        dcc.Dropdown(
            id='provinsi-dropdown',
            options=provinsi_options,
            value='jawa_barat',  
            multi=False, 
            placeholder="Pilih 1 provinsi",
            clearable=True,
            style={'width': '250px'}
        ),
        html.Label("Pilih Kejadian", style={'fontWeight': 'bold', 'marginBottom': '5px'}),
        dcc.Dropdown(
            id='kejadian-dropdown',
            options=kejadian_options,
            value=all_kejadian,  
            multi=True, 
            placeholder="Pilih Kejadian",
            clearable=False,
            style={'width': '250px'}
        ),
        html.Label("Tingkat Detail", style={'fontWeight': 'bold', 'marginBottom': '5px', 'marginTop': '10px'}),
        dcc.RadioItems(
            id='granularity-radio',
            options=[
                {'label': ' Provinsi', 'value': 'province'},
                {'label': ' Kabupaten/Kota', 'value': 'city'}
            ],
            value='province',
            style={'marginBottom': '10px'}
        ),
      html.Label("Rentang Tahun", style={'fontWeight': 'bold', 'marginBottom': '5px', 'marginTop': '10px'}),
            html.Div([
                dcc.Dropdown(
                    id='start-year-dropdown',
                    options=[{'label': str(year), 'value': year} for year in range(min_year, max_year + 1)],
                    value=min_year,
                    placeholder="Tahun Awal",
                    clearable=False,
                    style={'width': '110px', 'display': 'inline-block'}
                ),
                html.Span(" ", style={'margin': '0 5px', 'display': 'inline-block'}),
                dcc.Dropdown(
                    id='end-year-dropdown',
                    options=[{'label': str(year), 'value': year} for year in range(min_year, max_year + 1)],
                    value=max_year,
                    placeholder="Tahun Akhir",
                    clearable=False,
                    style={'width': '110px', 'display': 'inline-block'}
                ),
            ], style={'marginBottom': '10px'}),
    ]),
], style={
    "position": "absolute", 
    "top": "10px", 
    "left": "10px", 
    "zIndex": "9999",  
    "background": "white", 
    "padding": "15px", 
    "borderRadius": "5px",
    "boxShadow": "0 2px 5px rgba(0,0,0,0.2)",
    "pointerEvents": "auto"  
})

## Build App

In [22]:
# Create app
app = DashProxy(prevent_initial_callbacks=False)



# Set layout
app.layout = html.Div([
    # Controls outside the map
    controls,
    
    # Map
    dl.Map(
        children=[dl.TileLayer(), geojson, colorbar, info], 
        style={"height": "100vh", "width": "100vw"}, 
        center=[-2, 118],
        zoom=5,
        id="map"
    )
])


@app.callback(
    Output('geojson','data'),
    [Input("provinsi-dropdown","value"),
    Input("kejadian-dropdown","value"),
    Input("granularity-radio","value"),
     Input("start-year-dropdown","value"),  
     Input("end-year-dropdown","value")]   
)
def update_geojson(selected_provinsi, selected_kejadian, granularity, start_year, end_year):
    if not selected_provinsi:
        return {"type": "FeatureCollection", "features": []}
    
    provinsi_id = str(provinsi_dict[selected_provinsi])
    
    filtered_df = df[(df['ID Provinsi'] == int(provinsi_id)) & (df['Kejadian'].isin(selected_kejadian))]
    
    filtered_df = filtered_df[(filtered_df['Year'] >= start_year) & (filtered_df['Year'] <= end_year)]
    
    if granularity == 'province':
        density = len(filtered_df)
        return retrieve_instance_geojson([provinsi_id], {provinsi_id: density})
    
    else: 
        city_ids = province_city_map.get(provinsi_id, [])
        
        df_count = filtered_df.groupby('ID Kabupaten').size()
        density_dict = {str(k): v for k, v in df_count.to_dict().items()}
        
        complete_density_dict = {}
        for city_id in city_ids:
            complete_density_dict[city_id] = density_dict.get(city_id, 0)
        
        return retrieve_instance_geojson(city_ids, complete_density_dict)
    
@app.callback(
    Output("info", "children"), 
    Input("geojson", "hoverData")
)
def info_hover(feature):
    return get_info(feature)

## Run App

In [23]:
# Define color classes and scale
classes = [0, 1000, 2000, 3000, 4000, 5000, 6000, 7000]
colorscale = ["#FFEDA0", "#FED976", "#FEB24C", "#FD8D3C", "#FC4E2A", "#E31A1C", "#BD0026", "#800026"]
style = dict(weight=0, fillOpacity=0.8)

# Create colorbar
ctg = ["{}+".format(cls) for cls in classes[:-1]] + ["{}+".format(classes[-1])]
colorbar = dlx.categorical_colorbar(
    categories=ctg, 
    colorscale=colorscale, 
    width=300, 
    height=30, 
    position="bottomleft"
)

# Geojson rendering logic (JavaScript)
style_handle = assign("""function(feature, context){
    const {classes, colorscale, style, colorProp} = context.hideout;
    const value = feature.properties[colorProp];
    for (let i = 0; i < classes.length; ++i) {
        if (value > classes[i]) {
            style.fillColor = colorscale[i];
        }
    }
    return style;
}""")

In [24]:
if __name__ == "__main__":
    app.run(mode='inline', width='100%', height=800,prevent_initial_callbacks=False)